<a href="https://colab.research.google.com/github/govind1107/CNN/blob/main/KERAS_TUNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 97 kB 3.7 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
train_images =train_images/255
test_images = test_images/255


In [6]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [7]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [8]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [9]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [10]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 01m 22s]
val_accuracy: 0.8813333511352539

Best val_accuracy So Far: 0.9128333330154419
Total elapsed time: 00h 05m 07s
INFO:tensorflow:Oracle triggered exit


In [11]:
model=tuner_search.get_best_models(num_models=1)[0]

In [12]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 13s 7ms/step - loss: 0.1343 - accuracy: 0.9505 - val_loss: 0.2689 - val_accuracy: 0.9113
Epoch 5/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0940 - accuracy: 0.9650 - val_loss: 0.3071 - val_accuracy: 0.9117
Epoch 6/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0667 - accuracy: 0.9761 - val_loss: 0.3109 - val_accuracy: 0.9152
Epoch 7/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0474 - accuracy: 0.9828 - val_loss: 0.3638 - val_accuracy: 0.9100
Epoch 8/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0348 - accuracy: 0.9872 - val_loss: 0.4435 - val_accuracy: 0.9143
Epoch 9/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0311 - accuracy: 0.9893 - val_loss: 0.4789 - val_accuracy: 0.9123
Epoch 10/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0242 - accuracy: 0.9912 - val_loss: 0.4864 - val_a

In [14]:
print(tuner_search.get_best_models(num_models=1)[0])